In [1]:
import requests
import json
import time
import threading
import logging
from datetime import datetime, timedelta

In [2]:
logging.basicConfig(filename="cowin.log",
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.INFO)
logger = logging.getLogger("cowin")

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [4]:
states = json.loads(requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/states", headers=headers).text)['states']

In [5]:
states_info = {}
districts_rev = {}
for state in states:
    dists = json.loads(requests.get(f"https://cdn-api.co-vin.in/api/v2/admin/location/districts/{state['state_id']}", 
                                    headers=headers).text)['districts']
    districts = {}
    for d in dists:
        districts[d['district_id']] = d['district_name']
        districts_rev[d['district_name'].lower()] = d['district_id']
    states_info[state['state_id']] = {'name':state['state_name'], 'districts':districts}

In [6]:
states_rev = {s['state_name'].lower(): s['state_id'] for s in states}

In [7]:
logger.info(districts_rev)
logger.info(states_rev)

In [8]:
def find_appointments(district_ids, date, search_for_18_plus=True):
    result = []
    for district_id in district_ids:
        logger.info("Querying for district_id {} and date {}".format(district_id, date))
        url = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/calendarByDistrict?district_id={}&date={}".format(district_id, date)
        public_url = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(district_id, date)
        response = requests.get(url)
        if response.status_code != 200:
            response = requests.get(public_url, headers=headers)
        if response.status_code != 200:
            logger.error("No response received for district_id {} and date {}".format(district_id, date))
            continue
        appointments = json.loads(response.text)
        centers = appointments['centers']
        for center in centers:
            for session in center['sessions']:
                if session['available_capacity'] == 0:
                    continue
                if search_for_18_plus and session['min_age_limit'] > 18:
                    continue
                result.append("{} - {} - {} - {}".format(session['date'], center['name'],
                                                         center['district_name'], center['pincode']))
    return result

In [9]:
def valid_state(state):
    return state.lower() in states_rev.keys()

def valid_district(district):
    return district.lower() in districts_rev.keys()

In [10]:
!pip install python-telegram-bot --upgrade

     |████████████████████████████████| 455 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 5.9 MB/s  eta 0:00:01


In [11]:
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackContext,
)


In [12]:
STATE, DISTRICT, AGE = range(3)

In [13]:
def remove_job_if_exists(name: str, context: CallbackContext) -> bool:
    current_jobs = context.job_queue.get_jobs_by_name(name)
    if not current_jobs:
        return False
    for job in current_jobs:
        job.schedule_removal()
    return True

In [14]:
def start(update: Update, context: CallbackContext) -> int:
    remove_job_if_exists(str(update.message.chat_id), context)
    update.message.reply_text(
        'Hi! Im Cowin bot. I will help you book covid vaccination appointment near you. '
        'Send /stop anytime to stop talking to me.\n\n'
        'Which state are you in?\n'
        'Possible values:\n'+'\n'.join(states_rev.keys())
    )
    logger.info("{} has connected to the bot.".format(update.message.from_user['username']))
    return STATE

In [15]:
def state(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    chosen_state = update.message.text.strip()
    if not valid_state(chosen_state):
        update.message.reply_text('Please enter a valid state')
        return STATE
    
    logger.info("{} has selected state {}.".format(update.message.from_user['username'], chosen_state))
    state_id = states_rev[chosen_state.lower()]
    context.user_data["state"] = state_id
    possible_districts = list(states_info[state_id]['districts'].values())
    update.message.reply_text('Which district(s) you want to search for? Enter comma separated values.\nPossible values:\n{}'.format("\n".join(possible_districts)))
    return DISTRICT

In [16]:
def district(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    chosen_districts = update.message.text.split(",")
    chosen_districts = list(set([d.strip() for d in chosen_districts]))
    
    if not all(valid_district(district) for district in chosen_districts):
        update.message.reply_text('Please enter valid district(s).')
        return DISTRICT
    
    logger.info("{} has selected districts {}.".format(update.message.from_user['username'], chosen_districts))
    reply_keyboard = [['Yes', 'No']]
    district_ids = [districts_rev[d.lower()] for d in chosen_districts]
    context.user_data["districts"] = district_ids
    update.message.reply_text('Do you want to search only for slots available for 18-45 age group?',
                              reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
    return AGE

In [17]:
def callback_alarm(context: CallbackContext):
    user_data = context.job.context 
    appointments = find_appointments(user_data['districts'], datetime.today().strftime('%d-%m-%Y'), 
                                     user_data['only_18_plus'])
    if len(appointments) > 0:
        context.bot.send_message(chat_id=user_data['chat_id'], 
                                 text="The slots are available!! Centers -\n{}\nSend /stop anytime to stop talking to me.\n\n".format("\n".join(appointments[:20])))
        logger.info("Slots found for user {} searching for district_id {} and 18+ {}".format(
            user_data['username'], user_data['districts'], user_data["only_18_plus"]))

In [18]:
def age(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    only_18_plus = update.message.text.lower() == 'yes'
    logger.info("{} response for 18+ - {}.".format(user['username'], update.message.text))
    context.user_data["only_18_plus"] = only_18_plus
    chat_id = update.message.chat_id
    context.user_data.update({'chat_id':chat_id, 'username':user['username']})
    remove_job_if_exists(str(chat_id), context)
    context.job_queue.run_repeating(callback_alarm, 60, context=context.user_data, name=str(chat_id))
    update.message.reply_text('I will update you if any slots open up! Till then, take care and stay home if possible.', 
                              reply_markup=ReplyKeyboardRemove())
    return AGE

In [19]:
def stop(update: Update, context: CallbackContext) -> int:
    user = update.message.from_user
    update.message.reply_text(
        'Bye {}! Take care and stay home if possible.\n\nSend /start anytime to start talking to me.'.format(user.first_name), 
        reply_markup=ReplyKeyboardRemove()
    )
    remove_job_if_exists(str(update.message.chat_id), context)
    logger.info("{} has stopped the conversation.".format(update.message.from_user['username']))
    return ConversationHandler.END

In [ ]:
updater = Updater(token='1794674455:AAGjQarqlo_WFqopHyqVL-C7xCIkhjV2lAs')

dispatcher = updater.dispatcher

conv_handler = ConversationHandler(
    entry_points=[CommandHandler('start', start)],
    states={
        STATE: [MessageHandler(Filters.text & ~Filters.command, state)],
        DISTRICT: [MessageHandler(Filters.text & ~Filters.command, district)],
        AGE: [MessageHandler(Filters.regex('^(Yes|No)$'), age)]
    },
    fallbacks=[CommandHandler('stop', stop)],
)

dispatcher.add_handler(conv_handler)

updater.start_polling()

updater.idle()